In [1]:
from sklearn.base import BaseEstimator
import numpy as np

class SelfLearningModel(BaseEstimator):
    def __init__(self, basemodel, max_iter=200, prob_threshold=0.8):
        self.model = basemodel
        self.max_iter = max_iter
        self.prob_threshold = prob_threshold
        
    def fit(self, X, y):
        labeled_mask = y != -1
        labeled_X, labeled_y = X[labeled_mask], y[labeled_mask]
        unlabeled_X = X[~labeled_mask]
        self.model.fit(labeled_X, labeled_y)
        
        for _ in range(self.max_iter):
            unlabeled_y = self.model.predict(unlabeled_X)
            unlabeled_prob = np.max(self.model.predict_proba(unlabeled_X), axis=1)
            confident_indices = unlabeled_prob > self.prob_threshold
            if not np.any(confident_indices):
                break
            labeled_X = np.vstack([labeled_X, unlabeled_X[confident_indices]])
            labeled_y = np.hstack([labeled_y, unlabeled_y[confident_indices]])
            unlabeled_X = unlabeled_X[~confident_indices]
        return self
